In [ ]:
# 0.1. Cài đặt các thư viện cần thiết
!pip install -q fpdf2 noisereduce librosa tensorflow scikit-learn matplotlib seaborn pytz PyDrive2  

In [ ]:
# CÀI ĐẶT & CẤU HÌNH 
# 0.2. Import thư viện
import os
import glob 
import random
import datetime
import pytz
import shutil
import joblib
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from fpdf import FPDF
from tqdm import tqdm
import librosa
import noisereduce as nr
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from kaggle_secrets import UserSecretsClient
from oauth2client.service_account import ServiceAccountCredentials
from tensorflow.keras.regularizers import l2
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import roc_curve, auc as sklearn_auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from keras.saving import register_keras_serializable

print(f"TensorFlow Version: {tf.__version__}")

try:
    # --- 1. ƯU TIÊN KẾT NỐI VỚI TPU ---
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
    
    # Kiểu dữ liệu tối ưu nhất cho TPU là 'mixed_bfloat16'
    policy = 'mixed_bfloat16'
    mixed_precision.set_global_policy(policy)
    
    print(" KẾT NỐI TPU THÀNH CÔNG!")
    print(f"   - Số lượng nhân (replicas): {strategy.num_replicas_in_sync}")
    print(f"   - Kiểu dữ liệu (DType Policy): {policy}")

except Exception:
    print(" Không tìm thấy TPU. Đang kiểm tra GPU...")
    
    # --- 2. NẾU KHÔNG CÓ TPU, TÌM VÀ KẾT NỐI VỚI GPU ---
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        # MirroredStrategy sẽ tự động sử dụng TẤT CẢ các GPU tìm thấy
        strategy = tf.distribute.MirroredStrategy()
        
        # Kiểu dữ liệu tối ưu nhất cho GPU T4 là 'mixed_float16'
        policy = 'mixed_float16'
        mixed_precision.set_global_policy(policy)
        
        print(" KẾT NỐI GPU THÀNH CÔNG!")
        print(f"   - Số lượng GPU được sử dụng: {strategy.num_replicas_in_sync}")
        print(f"   - Kiểu dữ liệu (DType Policy): {policy}")
        
    else:
        # --- 3. NẾU KHÔNG CÓ CẢ GPU, SỬ DỤNG CPU ---
        print(" Không tìm thấy GPU. Sử dụng CPU.")
        strategy = tf.distribute.get_strategy()
        print(" Sử dụng chiến lược mặc định cho CPU.")
        print(f"   - Số lượng nhân (replicas): {strategy.num_replicas_in_sync}")


In [ ]:
# THIẾT LẬP CẤU HÌNH 
# --- Các cấu hình cơ bản ---
SEED = 42
def set_seed(seed_value):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
set_seed(SEED)

KAGGLE_PROCESSED_DATA_PATH = "/kaggle/input/ngt-spectrogram-id/"
KAGGLE_OUTPUT_PATH = "/kaggle/working/output_results"
CHECKPOINT_PATH = "/kaggle/working/checkpoints"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
os.makedirs(KAGGLE_OUTPUT_PATH, exist_ok=True)

CLASSES_TO_TRAIN = ['covid', 'asthma', 'healthy', 'tuberculosis']
ALL_CLASSES = ['healthy', 'asthma', 'covid', 'tuberculosis']
N_SPLITS = 5
TEST_SPLIT_RATIO = 0.15
USE_DATA_AUGMENTATION = True # Bật/tắt augmentation ở đây
USE_FOCAL_LOSS = True
USE_COSINE_DECAY_RESTARTS = True 

MODEL_ID = f'EfficienetV2B2_CV_TPU'
MIN_DELTA = 1e-4
SHUFFLE_BUFFER_SIZE = 2048 
GAMMA = 3.0 # Giữ nguyên giá trị tiêu chuẩn

# --- CÁC THAY ĐỔI CHÍNH ---
LEARNING_RATE = 1e-5              
WEIGHT_DECAY = 5e-4               

# Cấu hình cho Cross-Validation
TOTAL_EPOCHS = 500               
WARMUP_EPOCHS = 3                 
RESTART_CYCLE_1_EPOCHS = 50       
PATIENCE_EPOCHS = RESTART_CYCLE_1_EPOCHS + 25 # Thay đổi: Patience = 50

# --- ĐỊNH NGHĨA BATCH SIZE ---
# BATCH_SIZE này là batch size cho mỗi nhân TPU (per-replica)
BATCH_SIZE = 64
# Tính toán GLOBAL_BATCH_SIZE để dùng trong pipeline
# Biến 'strategy' được lấy từ ô code đầu tiên
GLOBAL_BATCH_SIZE = BATCH_SIZE * strategy.num_replicas_in_sync
print(f"Batch size mỗi nhân: {BATCH_SIZE}")
print(f"Global batch size (tổng cộng): {GLOBAL_BATCH_SIZE}")

# INPUT_SHAPE sẽ được cập nhật lại ở ô chuẩn bị dữ liệu
INPUT_SHAPE = (256, 256, 3)
print(f"Input shape: {INPUT_SHAPE}")

In [ ]:
# KHỞI TẠO CÁC HÀM CẦN THIẾT 

def get_patient_id(filepath, class_name):
    filename = os.path.basename(filepath)
    if class_name.lower() in ['asthma', 'covid', 'healthy']:
        return filename.split('_')[0]
    elif class_name.lower() == 'tuberculosis':
        return '_'.join(filename.split('_')[:-1]).replace('.npy', '')
    else:
        return filename.split('_')[0]

@register_keras_serializable()
class MacroF1Score(tf.keras.metrics.Metric):
    """
    Lớp metric để tính toán Macro F1-Score một cách chính xác trên toàn bộ epoch.
    """
    def __init__(self, num_classes, name='f1_macro', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight(name='tp', shape=(num_classes,), initializer='zeros')
        self.false_positives = self.add_weight(name='fp', shape=(num_classes,), initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', shape=(num_classes,), initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_labels = tf.argmax(tf.nn.softmax(y_pred), axis=1)
        y_true_labels = tf.argmax(y_true, axis=1)
        cm = tf.math.confusion_matrix(y_true_labels, y_pred_labels, num_classes=self.num_classes, dtype=tf.float32)
        tp = tf.linalg.diag_part(cm)
        fp = tf.reduce_sum(cm, axis=0) - tp
        fn = tf.reduce_sum(cm, axis=1) - tp
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        macro_f1 = tf.reduce_mean(f1)
        return macro_f1

    def reset_state(self):
        self.true_positives.assign(tf.zeros(self.num_classes))
        self.false_positives.assign(tf.zeros(self.num_classes))
        self.false_negatives.assign(tf.zeros(self.num_classes))

    # Thêm phương thức get_config
    def get_config(self):
        config = super(MacroF1Score, self).get_config()
        config.update({'num_classes': self.num_classes})
        return config

    def reset_state(self):
        # Reset các biến trạng thái về 0 ở đầu mỗi epoch
        self.true_positives.assign(tf.zeros(self.num_classes))
        self.false_positives.assign(tf.zeros(self.num_classes))
        self.false_negatives.assign(tf.zeros(self.num_classes))
        
def parse_tfrecord_fn(example):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    
    # 1. Parse tensor gốc (kích thước nhỏ)
    image = tf.io.parse_tensor(example['image'], out_type=tf.float32)
    
    # --- THÊM DÒNG SỬA LỖI TẠI ĐÂY ---
    # Thông báo cho TensorFlow biết hình dạng gốc của spectrogram là (256, 126)
    image.set_shape([256, 126])
    # ------------------------------------
    
    # 2. Thực hiện toàn bộ quá trình xử lý trên đồ thị TensorFlow
    image_3d = tf.stack([image, image, image], axis=-1)
    image_resized = tf.image.resize(image_3d, [INPUT_SHAPE[0], INPUT_SHAPE[1]])
    min_val = tf.reduce_min(image_resized)
    max_val = tf.reduce_max(image_resized)
    image_scaled_01 = (image_resized - min_val) / (max_val - min_val + 1e-7)
    image_scaled_255 = image_scaled_01 * 255.0
    
    # 3. Gọi hàm preprocess_input của model
    image_preprocessed = preprocess_input(image_scaled_255)
    
    # 4. Chuyển nhãn sang one-hot
    label = tf.one_hot(tf.cast(example['label'], tf.int32), depth=len(ALL_CLASSES))
    
    return image_preprocessed, label

def augment(spectrogram, label):
    spectrogram = spec_augment(spectrogram)
    return spectrogram, label

def focal_loss_from_logits_optimized(alpha, gamma=2.0):
    """
    Tạo ra hàm Focal Loss phiên bản đầy đủ và sạch sẽ.
    
    Args:
        alpha: Một list hoặc array chứa trọng số cho mỗi lớp.
        gamma: Hệ số tập trung, mặc định là 2.0.
    """
    # Chuyển alpha sang dạng tensor để tính toán
    alpha = tf.constant(alpha, dtype=tf.float32)

    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, 'float32')
        y_pred = tf.cast(y_pred, 'float32')
        
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
        probs = tf.nn.softmax(y_pred)
        pt = tf.reduce_sum(y_true * probs, axis=-1)
        focal_term = (1.0 - pt) ** gamma
        alpha_t = tf.reduce_sum(y_true * alpha, axis=-1)
        loss = alpha_t * focal_term * cross_entropy
        
        return tf.reduce_mean(loss)
        
    return focal_loss_fixed

def spec_augment(spectrogram, time_masking_para=40, frequency_masking_para=30, num_time_masks=1, num_freq_masks=1):
    spectrogram_aug = spectrogram
    freq_bins = tf.shape(spectrogram)[1] # Sửa: Lấy chiều tần số từ shape 4D
    time_steps = tf.shape(spectrogram)[2] # Sửa: Lấy chiều thời gian từ shape 4D
    
    for _ in range(num_freq_masks):
        f = tf.random.uniform(shape=(), minval=0, maxval=frequency_masking_para, dtype=tf.int32)
        f0 = tf.random.uniform(shape=(), minval=0, maxval=freq_bins - f, dtype=tf.int32)
        freq_mask_1d = tf.concat([tf.ones((f0,), dtype=spectrogram.dtype), tf.zeros((f,), dtype=spectrogram.dtype), tf.ones((freq_bins - f0 - f,), dtype=spectrogram.dtype)], axis=0)
        freq_mask_4d = tf.reshape(freq_mask_1d, (1, freq_bins, 1, 1)) # Sửa: Reshape thành 4D để broadcast
        spectrogram_aug *= freq_mask_4d
        
    for _ in range(num_time_masks):
        t = tf.random.uniform(shape=(), minval=0, maxval=time_masking_para, dtype=tf.int32)
        t0 = tf.random.uniform(shape=(), minval=0, maxval=time_steps - t, dtype=tf.int32)
        time_mask_1d = tf.concat([tf.ones((t0,), dtype=spectrogram.dtype), tf.zeros((t,), dtype=spectrogram.dtype), tf.ones((time_steps - t0 - t,), dtype=spectrogram.dtype)], axis=0)
        time_mask_4d = tf.reshape(time_mask_1d, (1, 1, time_steps, 1)) # Sửa: Reshape thành 4D để broadcast
        spectrogram_aug *= time_mask_4d
        
    return spectrogram_aug

@register_keras_serializable()
class FinalModel(tf.keras.Model):
    def __init__(self, input_shape, num_classes, **kwargs):
        super(FinalModel, self).__init__(**kwargs)
        self.input_shape_config = input_shape
        self.num_classes_config = num_classes
        
        # 1. Mô hình nền 
        self.base_model = EfficientNetV2B2(
            weights='imagenet',
            include_top=False,
            input_shape=self.input_shape_config
        )
        
        # 2. Các lớp "Head" phân loại phức tạp hơn
        self.pooling = GlobalAveragePooling2D(name="pooling_layer")
        self.dense1 = Dense(512, activation='relu', kernel_regularizer=l2(0.001), name="dense_layer_1")
        self.dropout1 = Dropout(0.5, name="dropout_layer_1")
        self.dense2 = Dense(256, activation='relu', kernel_regularizer=l2(0.001), name="dense_layer_2")
        self.dropout2 = Dropout(0.3, name="dropout_layer_2")
        self.dense_output = Dense(num_classes, activation='linear', dtype='float32', name="output_layer")

    def call(self, inputs, training=None):
        # Dữ liệu đi qua mô hình nền
        x = self.base_model(inputs, training=training)
        
        # Dữ liệu đi qua các lớp "Head" mới
        x = self.pooling(x)
        x = self.dense1(x)
        x = self.dropout1(x, training=training)
        x = self.dense2(x)
        x = self.dropout2(x, training=training)
        outputs = self.dense_output(x)
        return outputs

    # Phương thức get_config (giữ nguyên)
    def get_config(self):
        config = super(FinalModel, self).get_config()
        config.update({
            'input_shape': self.input_shape_config,
            'num_classes': self.num_classes_config
        })
        return config

def load_data_from_df(df):
    X, y = [], []
    for _, row in df.iterrows():
        X.append(np.load(row['filepath']))
        y.append(row['label'])
    return np.array(X), np.array(y)

def get_grad_cam_final(model, img_array, last_conv_layer_name, pred_index=None):
    """
    Tạo Grad-CAM cho một subclassed model.
    Lưu ý: Model phải được build (chạy qua dữ liệu một lần) trước khi gọi hàm này.
    """
    # Tạo một model trung gian với input là input của model chính,
    # và output là lớp conv cuối và output cuối cùng của model chính.
    grad_model = Model(
        inputs=model.inputs,
        outputs=[model.base_model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Tính toán gradient
    with tf.GradientTape() as tape:
        # Đưa ảnh vào grad_model để lấy 2 output đã định nghĩa ở trên
        last_conv_layer_output, preds = grad_model(img_array)
        
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Lấy gradient của lớp được dự đoán đối với feature map của lớp conv cuối
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Tính trung bình gradient và tạo heatmap
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + tf.keras.backend.epsilon())
    
    return heatmap.numpy()

def overlay_grad_cam(spec, heatmap, alpha=0.6):
    heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (spec.shape[0], spec.shape[1]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    jet = plt.cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap_resized.squeeze()]
    spec_display = np.stack([spec]*3, axis=-1)
    spec_display = (spec_display - spec_display.min()) / (spec_display.max() - spec_display.min())
    superimposed_img = jet_heatmap * alpha + spec_display
    superimposed_img = np.clip(superimposed_img, 0, 1)
    return superimposed_img

class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'BAO CAO KET QUA HUAN LUYEN MO HINH AI', 0, 1, 'C')
        self.ln(10)
    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Trang {self.page_no()}', 0, 0, 'C')
    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)
    def chapter_body(self, content):
        self.set_font('Arial', '', 10)
        safe_content = content.encode('latin-1', 'replace').decode('latin-1')
        self.multi_cell(0, 5, safe_content)
        self.ln()
    def add_image_section(self, title, img_path):
        self.chapter_title(title)
        if os.path.exists(img_path):
            self.image(img_path, x=None, y=None, w=180)
            self.ln(5)
        else:
            self.chapter_body(f"Khong tim thay hinh anh: {img_path}")

def authenticate_gdrive():
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("google_service_account_key")
    with open("service_account.json", "w") as f:
        f.write(secret_value)
    scope = ["https://www.googleapis.com/auth/drive"]
    gauth = GoogleAuth()
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    drive = GoogleDrive(gauth)
    return drive

def upload_folder_to_drive(drive, folder_path, parent_folder_id):
    folder_name = os.path.basename(folder_path)
    print(f"Đang tạo thư mục '{folder_name}' trên Google Drive...")
    folder_metadata = {'title': folder_name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [{'id': parent_folder_id}]}
    folder = drive.CreateFile(folder_metadata)
    folder.Upload()
    
    print(f"Bắt đầu tải nội dung của '{folder_name}'...")
    for item in tqdm(os.listdir(folder_path), desc=f"Uploading {folder_name}"):
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path):
            gfile = drive.CreateFile({'title': item, 'parents': [{'id': folder['id']}]})
            gfile.SetContentFile(item_path)
            gfile.Upload(param={'supportsTeamDrives': True})
        elif os.path.isdir(item_path):
            upload_folder_to_drive(drive, item_path, folder['id'])

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(image, label):
    """Creates a tf.train.Example message ready to be written to a file."""
    feature = {
        'image': _bytes_feature(tf.io.serialize_tensor(image)),
        'label': _int64_feature(label)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        current_lr = tf.keras.backend.get_value(self.model.optimizer.learning_rate)
        print(f"\nEpoch {epoch+1}: Learning Rate is {current_lr:.2e}")

In [ ]:
# CHUẨN BỊ DỮ LIỆU VÀ TẠO TFRECORD 

suspicious_files_to_remove = [
    '/kaggle/input/ngt-spectrogram-id/healthy/P0030101_123370_Dkwg3F7jMGaR7kbc-seg2.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0032202_15897_PcbyJQWemBfghUYp-seg2.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0027142_5701_hupBI5CxKMNCfe8b-seg1.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0056214_89533_0WsmNRSKuQFGodg1-seg1.npy',
]
# --- BƯỚC 1: TẢI VÀ PHÂN CHIA DỮ LIỆU BAN ĐẦU ---
print("Bắt đầu chuẩn bị và phân chia dữ liệu...")
all_files_to_split = []
for class_name in ALL_CLASSES:
    source_dir = os.path.join(KAGGLE_PROCESSED_DATA_PATH, class_name)
    if os.path.exists(source_dir):
        files = glob.glob(os.path.join(source_dir, '*.npy'))
        for f in files:
            all_files_to_split.append({'filepath': f, 'label': class_name})

all_data_df = pd.DataFrame(all_files_to_split)
all_data_df['patient_id'] = all_data_df.apply(lambda row: get_patient_id(row['filepath'], row['label']), axis=1)

print(f"Số lượng mẫu ban đầu: {len(all_data_df)}")
all_data_df = all_data_df[~all_data_df['filepath'].isin(suspicious_files_to_remove)].reset_index(drop=True)
print(f"Số lượng mẫu sau khi lọc bỏ file 'im lặng': {len(all_data_df)}")

print("Tách tập Test cuối cùng (Hold-out set)...")
patient_ids = all_data_df['patient_id'].unique()
np.random.shuffle(patient_ids)
test_patient_count = int(len(patient_ids) * TEST_SPLIT_RATIO)
test_patients = patient_ids[:test_patient_count]
train_val_patients = patient_ids[test_patient_count:]

test_df = all_data_df[all_data_df['patient_id'].isin(test_patients)].reset_index(drop=True)
train_val_df = all_data_df[all_data_df['patient_id'].isin(train_val_patients)].reset_index(drop=True)

print(f"Đã tách: {len(train_val_df)} mẫu cho Train/Validation (CV) và {len(test_df)} mẫu cho Test cuối cùng.")

# --- BƯỚC 2: KHỞI TẠO LABEL ENCODER ---
le = LabelEncoder().fit(ALL_CLASSES)

# --- BƯỚC 3: CHỈ TẠO FILE TFRECORD CHO TẬP TEST ---
TFRECORD_OUTPUT_PATH = "/kaggle/working/tfrecords"
os.makedirs(TFRECORD_OUTPUT_PATH, exist_ok=True)
print(f"Bắt đầu chuyển đổi dữ liệu sang TFRecord tại: {TFRECORD_OUTPUT_PATH}")

print("--- Đang xử lý và tạo file cho tập test ---")
test_tfrecord_path = os.path.join(TFRECORD_OUTPUT_PATH, "test.tfrec")
with tf.io.TFRecordWriter(test_tfrecord_path) as writer:
    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Creating test.tfrec"):
        spectrogram = np.load(row['filepath']).astype(np.float32)
        
        image_tensor = tf.convert_to_tensor(spectrogram)
        image_3d = tf.stack([image_tensor]*3, axis=-1)
        image_resized = tf.image.resize(image_3d, [INPUT_SHAPE[0], INPUT_SHAPE[1]])
        min_val = tf.reduce_min(image_resized)
        max_val = tf.reduce_max(image_resized)
        image_scaled_01 = (image_resized - min_val) / (max_val - min_val + 1e-7)
        image_to_serialize = image_scaled_01 * 255.0

        label_encoded = le.transform([row['label']])[0]
        example = serialize_example(image_to_serialize, label_encoded)
        writer.write(example)

print("\\nChuẩn bị dữ liệu ban đầu hoàn tất!")

In [ ]:
# HUẤN LUYỆN MÔ HÌNH Cross-Validation

# --- BƯỚC 1: KHỞI TẠO CÁC BIẾN CẦN THIẾT ---
print("Đang khởi tạo các biến cho Cross-Validation...")
AUTOTUNE = tf.data.AUTOTUNE
skf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
# Sử dụng train_val_df đã được tạo ở ô trước
y_labels_for_split = le.transform(train_val_df['label'])
groups_for_split = train_val_df['patient_id'].values

fold_accuracies, fold_losses, fold_aucs, fold_f1s = [], [], [], []

print("Đang tính toán trọng số alpha cho Focal Loss...")
class_weights_array = class_weight.compute_class_weight('balanced', classes=np.unique(y_labels_for_split), y=y_labels_for_split)
alpha_weights_list = class_weights_array.tolist()
print("Trọng số Alpha được tính toán:")
for i, w in enumerate(alpha_weights_list):
    class_name = le.inverse_transform([i])[0]
    print(f"- Lớp '{class_name}': {w:.2f}")

# --- BƯỚC 2: BẮT ĐẦU VÒNG LẶP CROSS-VALIDATION ---
for fold, (train_indices, val_indices) in enumerate(skf.split(train_val_df, y_labels_for_split, groups_for_split)):
    fold_number = fold + 1
    print("-" * 50 + f"\\nBắt đầu Fold {fold_number}/{N_SPLITS}\\n" + "-" * 50)

    checkpoint_filepath = os.path.join(CHECKPOINT_PATH, f'fold_{fold_number}_checkpoint.keras')
    # Tạo model hoặc tải lại từ checkpoint
    with strategy.scope():
        if os.path.exists(checkpoint_filepath):
            print(f"--- Tìm thấy checkpoint, đang tải lại model từ: {checkpoint_filepath} ---")
            model = tf.keras.models.load_model(checkpoint_filepath, custom_objects={...}) # Điền custom objects
        else:
            print("--- Không tìm thấy checkpoint, tạo model mới ---")
            model = FinalModel(input_shape=INPUT_SHAPE, num_classes=len(ALL_CLASSES))
            
    # === TẠO FILE TFRECORD "JUST-IN-TIME" VỚI DỮ LIỆU THÔ ===
    print(f"--- Đang tạo file TFRecord (dữ liệu thô) cho Fold {fold_number} ---")
    train_fold_df = train_val_df.iloc[train_indices]
    val_fold_df = train_val_df.iloc[val_indices]
    
    train_tfrec_path = os.path.join(TFRECORD_OUTPUT_PATH, f"train_fold_{fold_number}.tfrec")
    val_tfrec_path = os.path.join(TFRECORD_OUTPUT_PATH, f"val_fold_{fold_number}.tfrec")

    # Hàm trợ giúp để ghi dữ liệu THÔ, chưa qua xử lý
    def write_raw_tfrecord(df, path, desc):
        with tf.io.TFRecordWriter(path) as writer:
            for _, row in tqdm(df.iterrows(), total=len(df), desc=desc):
                # Chỉ load và serialize spectrogram gốc, không xử lý gì thêm
                spectrogram = np.load(row['filepath']).astype(np.float32)
                label_encoded = le.transform([row['label']])[0]
                example = serialize_example(spectrogram, label_encoded)
                writer.write(example)
    
    write_raw_tfrecord(train_fold_df, train_tfrec_path, f"Writing Raw Train Fold {fold_number}")
    write_raw_tfrecord(val_fold_df, val_tfrec_path, f"Writing Raw Val Fold {fold_number}")
    print(f"--- Đã tạo xong file TFRecord (dữ liệu thô) cho Fold {fold_number} ---")
    
    # Đọc trực tiếp file vừa tạo
    train_ds = tf.data.TFRecordDataset(train_tfrec_path)
    val_ds = tf.data.TFRecordDataset(val_tfrec_path)
    train_ds = train_ds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).repeat().batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    
    steps_per_epoch = len(train_indices) // GLOBAL_BATCH_SIZE
    validation_steps = len(val_indices) // GLOBAL_BATCH_SIZE
    print(f"Số bước mỗi epoch: {steps_per_epoch} | Số bước validation: {validation_steps}")


    # --- TẠO MODEL VÀ LOSS FUNCTION ---
    with strategy.scope():
        model = FinalModel(input_shape=INPUT_SHAPE, num_classes=len(ALL_CLASSES))
        if USE_FOCAL_LOSS:
            loss_function = focal_loss_from_logits_optimized(alpha=alpha_weights_list, gamma=GAMMA)
        else:
            loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # --- GIAI ĐOẠN 1: HUẤN LUYỆN CÁC LỚP CUỐI (PHIÊN BẢN TỐI ƯU) ---
    print("\\n--- Bắt đầu Giai đoạn 1: Huấn luyện các lớp cuối ---")
    
    # Tạo callback EarlyStopping chỉ dành riêng cho giai đoạn này
    head_early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Dừng lại nếu val_loss không cải thiện sau 5 epochs
        restore_best_weights=True,
        verbose=1
    )

    with strategy.scope():
        # Đóng băng toàn bộ mô hình nền
        model.base_model.trainable = False
        
        # Compile với learning rate lớn hơn cho head
        optimizer_head = tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=WEIGHT_DECAY)
        model.compile(optimizer=optimizer_head, 
                      loss=loss_function, 
                      metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
        
        print("Bắt đầu huấn luyện head với Early Stopping...")
        # Huấn luyện cho tối đa 20 epochs, nhưng sẽ dừng sớm nếu cần
        model.fit(train_ds, 
                  validation_data=val_ds, 
                  epochs=20,  # Tăng số epochs tối đa
                  steps_per_epoch=steps_per_epoch, 
                  validation_steps=validation_steps, 
                  callbacks=[head_early_stopping], # Thêm callback vào đây
                  verbose=1)

        # 2. GIAI ĐOẠN 2A: WARMUP
        print(f"\n--- Giai đoạn 2A: Bắt đầu Warmup trong {WARMUP_EPOCHS} epochs ---")
        warmup_lr = LEARNING_RATE / 10 # Bắt đầu với LR rất thấp
        f1_macro = MacroF1Score(num_classes=len(ALL_CLASSES), name='f1_macro')
        optimizer_warmup = tf.keras.optimizers.AdamW(learning_rate=warmup_lr, weight_decay=WEIGHT_DECAY)
        model.compile(optimizer=optimizer_warmup, loss=loss_function, metrics=['accuracy', f1_macro])

        model.fit(
            train_ds, validation_data=val_ds, epochs=WARMUP_EPOCHS,
            steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, verbose=1
        )

        # 3. GIAI ĐOẠN 2B: HUẤN LUYỆN CHÍNH VỚI COSINE DECAY RESTARTS
        print(f"\n--- Giai đoạn 2B: Bắt đầu huấn luyện chính ---")

        # Sử dụng công tắc để chọn optimizer và callbacks
        if USE_COSINE_DECAY_RESTARTS:
            print("Sử dụng scheduler: CosineDecayRestarts")
            first_decay_steps = RESTART_CYCLE_1_EPOCHS * steps_per_epoch
            lr_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(
                initial_learning_rate=LEARNING_RATE,
                first_decay_steps=first_decay_steps,
                t_mul=2.0, 
                m_mul=0.9,  
                alpha=0.1
            )
            optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=lr_scheduler, weight_decay=WEIGHT_DECAY)
            
            # Callbacks cho CosineDecayRestarts
            callbacks = [
                EarlyStopping(
                    monitor='val_f1_macro', mode='max', patience=PATIENCE_EPOCHS,
                    restore_best_weights=True, min_delta=MIN_DELTA, verbose=1
                ),
                LearningRateLogger()
            ]
        else:
            print("Sử dụng scheduler: ReduceLROnPlateau")
            # Optimizer với learning rate ban đầu cố định
            optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
            
            # Callbacks cho ReduceLROnPlateau
            callbacks = [
                EarlyStopping(
                    monitor='val_f1_macro', mode='max', patience=30, # Có thể cần patience dài hơn
                    restore_best_weights=True, min_delta=MIN_DELTA, verbose=1
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_f1_macro', mode='max', factor=0.2,
                    patience=10, # Giảm LR nếu F1 không cải thiện trong 10 epochs
                    min_lr=1e-7,
                    verbose=1
                ),
                LearningRateLogger()
            ]

        # Biên dịch lại mô hình với optimizer và metrics cuối cùng
        with strategy.scope():
            model.compile(optimizer=optimizer_finetune, loss=loss_function, metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), f1_macro])
        
        # Huấn luyện mô hình
        history = model.fit(
            train_ds, validation_data=val_ds, epochs=TOTAL_EPOCHS,
            initial_epoch=WARMUP_EPOCHS,
            steps_per_epoch=steps_per_epoch, validation_steps=validation_steps,
            callbacks=callbacks, verbose=1
        )

    # Tạo đường dẫn và tên file để lưu trọng số
    weights_save_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.weights.h5')
    # Chỉ lưu lại trọng số của mô hình
    model.save_weights(weights_save_path)
    # In ra thông báo để xác nhận
    print(f"Đã lưu trọng số cho Fold {fold_number} tại: {weights_save_path}")

    # --- VẼ BIỂU ĐỒ VÀ ĐÁNH GIÁ ---
    print("Đang tạo và lưu biểu đồ huấn luyện...")
    plt.figure(figsize=(18, 7))
    plt.suptitle(f'Training Metrics for Fold {fold_number}', fontsize=16)
    
    # --- Biểu đồ cho các chỉ số (Accuracy, AUC, F1-Macro) ---
    plt.subplot(1, 2, 1)
    # Vẽ các chỉ số của tập Train
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linestyle='-')
    plt.plot(history.history['auc'], label='Training AUC', color='green', linestyle='-')
    plt.plot(history.history['f1_macro'], label='Training F1-Macro', color='red', linestyle='-')
    
    # Vẽ các chỉ số của tập Validation
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='blue', linestyle='--')
    plt.plot(history.history['val_auc'], label='Validation AUC', color='green', linestyle='--')
    plt.plot(history.history['val_f1_macro'], label='Validation F1-Macro', color='red', linestyle='--')
    
    # Cập nhật lại tiêu đề và nhãn
    plt.title('Biểu đồ các chỉ số (Accuracy, AUC, F1-Macro)')
    plt.xlabel('Epoch')
    plt.ylabel('Giá trị')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    # --- Biểu đồ cho Loss ---
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', color='orange')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='purple')
    plt.title('Biểu đồ Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    # Lưu và đóng hình ảnh
    plot_filename = f'fold_{fold_number}_metrics.png'
    plot_filepath = os.path.join(KAGGLE_OUTPUT_PATH, plot_filename)
    plt.savefig(plot_filepath)
    plt.close()
    
    print(f"Đã lưu biểu đồ cho Fold {fold_number} tại: {plot_filepath}")
    
    # THÊM 4: Sửa lại model.evaluate để nhận đủ 4 giá trị
    loss, accuracy, auc, f1 = model.evaluate(val_ds, verbose=0)
    print(f"Fold {fold_number} - Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}, Validation AUC: {auc:.4f}, Validation F1-Macro: {f1:.4f}")
    
    fold_aucs.append(auc)
    fold_losses.append(loss)
    fold_accuracies.append(accuracy)
    fold_f1s.append(f1) # Thêm lưu trữ F1

    # THÊM 5: Sửa lại câu lệnh print cuối cùng
    print("=" * 50 + "\nKết quả Cross-Validation:\n" 
      + f"Validation Accuracy trung bình: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}\n"
      + f"Validation Loss trung bình: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}\n"
      + f"Validation AUC trung bình: {np.mean(fold_aucs):.4f} +/- {np.std(fold_aucs):.4f}\n"
      + f"Validation F1-Macro trung bình: {np.mean(fold_f1s):.4f} +/- {np.std(fold_f1s):.4f}\n" # Thêm dòng này
      + "=" * 50)
    print(f"\\n--- Dọn dẹp file cho Fold {fold_number} ---")
    try:
        os.remove(train_tfrec_path)
        os.remove(val_tfrec_path)
        print(f"Đã xóa thành công file tạm của Fold {fold_number}")
    except OSError as e:
        print(f"Lỗi khi xóa file: {e}")

In [ ]:
# --- Bắt đầu quy trình đánh giá tổng hợp 5-Fold trên tập test ---

print("--- Chuẩn bị dữ liệu Test cho việc đánh giá ---")

# 1. Tạo Test Dataset từ file TFRecord đã được xử lý trước
TEST_TFREC_PATH = os.path.join(TFRECORD_OUTPUT_PATH, "test.tfrec")
if not os.path.exists(TEST_TFREC_PATH):
    print(f"Lỗi: Không tìm thấy file test.tfrec tại {TEST_TFREC_PATH}. Vui lòng chạy lại ô chuẩn bị dữ liệu.")
else:
    # Tạo dataset đã được batch để đưa vào model.evaluate
    test_ds_batched = tf.data.TFRecordDataset(TEST_TFREC_PATH)
    test_ds_batched = test_ds_batched.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    test_ds_batched = test_ds_batched.batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    # --- Bắt đầu quá trình đánh giá ---
    evaluation_results = []
    output_filename = "teacher_models_evaluation_summary.csv"
    output_filepath = os.path.join(KAGGLE_OUTPUT_PATH, output_filename)
    n_classes = len(ALL_CLASSES)

    # Vòng lặp qua 5 Fold để tải và đánh giá từng mô hình
    for fold_number in range(1, N_SPLITS + 1):
        print(f"\\n---> Đang đánh giá Fold {fold_number}/{N_SPLITS}...")
        
        model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        
        if not os.path.exists(model_path):
            print(f"!!! Cảnh báo: Không tìm thấy file model cho Fold {fold_number} tại '{model_path}'. Bỏ qua fold này.")
            continue

        try:
            with strategy.scope():
                # Tải lại mô hình đã huấn luyện
                model = tf.keras.models.load_model(
                    model_path,
                    custom_objects={
                        'focal_loss_fixed': focal_loss_from_logits_optimized(alpha=alpha_weights_list),
                        'MacroF1Score': MacroF1Score
                    }
                )
                
                # Biên dịch lại mô hình để đảm bảo các metrics được tính đúng
                model.compile(
                    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                    optimizer='adam', # Optimizer ở đây không quan trọng vì không huấn luyện lại
                    metrics=[
                        'accuracy', 
                        tf.keras.metrics.AUC(name='auc'), 
                        MacroF1Score(num_classes=n_classes, name='f1_macro')
                    ]
                )

            # Đánh giá mô hình trên tập Test đã được batch
            print(f"Bắt đầu tính toán các chỉ số cho Fold {fold_number}...")
            results = model.evaluate(test_ds_batched, return_dict=True, verbose=0)
            
            # Lưu kết quả của fold hiện tại
            fold_summary = {
                'Fold': fold_number,
                'Loss': results.get('loss', 0),
                'Accuracy': results.get('accuracy', 0),
                'AUC': results.get('auc', 0),
                'F1-Macro': results.get('f1_macro', 0)
            }
            evaluation_results.append(fold_summary)
            print(f"Kết quả Fold {fold_number}: {fold_summary}")

        except Exception as e:
            print(f"!!! Lỗi khi xử lý Fold {fold_number}: {e}")

    # --- Tổng hợp và Lưu kết quả ra file CSV ---
    if evaluation_results:
        results_df = pd.DataFrame(evaluation_results)
        
        # Tính toán giá trị trung bình và độ lệch chuẩn
        summary_stats = results_df.drop('Fold', axis=1).agg(['mean', 'std'])
        print("\\n\\n--- Thống kê tổng hợp (5 Folds) trên tập Test ---")
        print(summary_stats)
        
        # Tạo dòng tổng kết để thêm vào file CSV
        summary_stats_df = summary_stats.reset_index().rename(columns={'index': 'Fold'})
        
        # Ghép kết quả chi tiết và kết quả tổng hợp
        final_df_to_save = pd.concat([results_df, summary_stats_df], ignore_index=True)
        
        final_df_to_save.to_csv(output_filepath, index=False, float_format='%.4f')
        print(f"\\n--- HOÀN TẤT ---")
        print(f"Bảng kết quả tổng hợp đã được lưu tại: {output_filepath}")
        
        print("\\nNội dung file CSV:")
        print(final_df_to_save.to_string(index=False))
    else:
        print("\\nKhông có fold nào được đánh giá thành công. Không có file CSV nào được tạo.")

In [ ]:
# --- Bắt đầu quy trình vẽ đường cong ROC trung bình 5-Fold ---

print("--- Chuẩn bị dữ liệu Test cho việc vẽ ROC ---")

# 1. Tạo Test Dataset và trích xuất nhãn thật
TEST_TFREC_PATH = os.path.join(TFRECORD_OUTPUT_PATH, "test.tfrec")
if not os.path.exists(TEST_TFREC_PATH):
    print(f"Lỗi: Không tìm thấy file test.tfrec tại {TEST_TFREC_PATH}. Vui lòng chạy lại ô chuẩn bị dữ liệu.")
else:
    # Tạo dataset gốc để lấy nhãn
    test_ds_unbatched = tf.data.TFRecordDataset(TEST_TFREC_PATH)
    test_ds_unbatched = test_ds_unbatched.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    
    # Tạo dataset đã được batch để lấy dự đoán
    test_ds_batched = test_ds_unbatched.batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    print("Đang trích xuất nhãn từ tập test...")
    y_test_onehot = np.concatenate([y.numpy() for x, y in test_ds_unbatched], axis=0)
    print(f"Đã trích xuất xong {len(y_test_onehot)} nhãn.")

    # --- Bắt đầu quá trình lấy dự đoán và vẽ biểu đồ ---
    y_test_binarized = y_test_onehot
    n_classes = y_test_binarized.shape[1]
    class_names = le.classes_
    
    all_y_preds_probs = []
    print("\\nĐang tải 5 model và lấy dự đoán từ 5 folds...")
    for fold_number in tqdm(range(1, N_SPLITS + 1), desc="Processing Folds for ROC"):
        model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        if not os.path.exists(model_path):
            continue
        
        with strategy.scope():
            model = tf.keras.models.load_model(
                model_path,
                custom_objects={
                    'focal_loss_fixed': focal_loss_from_logits_optimized(alpha=alpha_weights_list),
                    'MacroF1Score': MacroF1Score
                }
            )
        
        all_logits = []
        for images_batch, _ in test_ds_batched:
            batch_logits = model(images_batch, training=False)
            all_logits.append(batch_logits.numpy())
        
        y_pred_logits = np.concatenate(all_logits, axis=0)
        y_pred_probs = tf.nn.softmax(y_pred_logits).numpy()
        all_y_preds_probs.append(y_pred_probs)

    # --- Tính toán ROC và nội suy ---
    print("\\nĐang tính toán ROC và nội suy...")
    tprs_per_class = [[] for _ in range(n_classes)]
    aucs_per_class = [[] for _ in range(n_classes)]
    mean_fpr = np.linspace(0, 1, 100)

    for y_pred_probs in all_y_preds_probs:
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_probs[:, i])
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs_per_class[i].append(interp_tpr)
            aucs_per_class[i].append(sklearn_auc(fpr, tpr))

    # --- Vẽ biểu đồ ---
    print("Đang vẽ biểu đồ...")
    plt.figure(figsize=(12, 10))
    colors = plt.cm.get_cmap('tab10', n_classes)

    for i in range(n_classes):
        mean_tpr = np.mean(tprs_per_class[i], axis=0)
        mean_tpr[-1] = 1.0
        std_tpr = np.std(tprs_per_class[i], axis=0)
        mean_auc = np.mean(aucs_per_class[i])
        std_auc = np.std(aucs_per_class[i])

        plt.plot(mean_fpr, mean_tpr, color=colors(i),
                 label=f'Lớp: {class_names[i]} (AUC = {mean_auc:.2f} $\\pm$ {std_auc:.2f})',
                 lw=2)
        
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color=colors(i), alpha=.2)

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Đường cong ROC trung bình (5-Fold Cross-Validation)', fontsize=16)
    plt.legend(loc="lower right", fontsize=12)
    plt.grid(True)
    
    output_filename = "roc_curve_5_folds_average.png"
    output_filepath = os.path.join(KAGGLE_OUTPUT_PATH, output_filename)
    plt.savefig(output_filepath, dpi=300)
    print(f"\\nHoàn tất! Biểu đồ đã được lưu tại: {output_filepath}")
    
    plt.show()

In [ ]:
# --- Bắt đầu quy trình phân tích Grad-CAM chi tiết cho 5 Folds ---

print("--- Chuẩn bị dữ liệu Test cho việc phân tích Grad-CAM ---")

# 1. Tạo Test Dataset từ file TFRecord đã được xử lý trước
# Lưu ý: Các biến TFRECORD_OUTPUT_PATH, parse_tfrecord_fn, GLOBAL_BATCH_SIZE, AUTOTUNE
# đã được định nghĩa ở các ô code phía trên.
TEST_TFREC_PATH = os.path.join(TFRECORD_OUTPUT_PATH, "test.tfrec")
if not os.path.exists(TEST_TFREC_PATH):
    print(f"Lỗi: Không tìm thấy file test.tfrec tại {TEST_TFREC_PATH}. Vui lòng chạy lại ô chuẩn bị dữ liệu.")
else:
    test_ds_batched = tf.data.TFRecordDataset(TEST_TFREC_PATH)
    test_ds_batched = test_ds_batched.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    test_ds_batched = test_ds_batched.batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    # --- Định nghĩa các hàm cần thiết cho Grad-CAM ---
    target_names = le.classes_
    grad_cam_main_path = os.path.join(KAGGLE_OUTPUT_PATH, "grad_cam_detailed_analysis")
    os.makedirs(grad_cam_main_path, exist_ok=True)

    def find_last_conv_layer(model):
        for layer in reversed(model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                return layer
            if isinstance(layer, tf.keras.Model):
                return find_last_conv_layer(layer)
        return None

    @tf.function
    def get_grad_cam_batched(model, img_batch):
        if hasattr(model, 'base_model'):
            inp = model.base_model.input
            last_conv_layer = model.base_model.get_layer("top_conv")
            x = model.pooling(model.base_model.output)
            x = model.dense1(x)
            if hasattr(model, 'dense2'): # Thích ứng với head phức tạp hơn
                x = model.dense2(x)
            final_output = model.dense_output(x)
            grad_model = Model(inp, [last_conv_layer.output, final_output])
        else:
            last_conv_layer = find_last_conv_layer(model)
            grad_model = Model([model.inputs], [last_conv_layer.output, model.output])
        
        with tf.GradientTape() as tape:
            last_conv_layer_output_value, preds = grad_model(img_batch)
            pred_indices = tf.argmax(preds, axis=1)
            class_channels = tf.gather(preds, pred_indices, axis=1, batch_dims=1)

        grads = tape.gradient(class_channels, last_conv_layer_output_value)
        pooled_grads = tf.reduce_mean(grads, axis=(1, 2))
        heatmap_batch = tf.einsum('bhwc,bc->bhw', last_conv_layer_output_value, pooled_grads)
        heatmap_batch = tf.maximum(heatmap_batch, 0)
        max_vals = tf.reduce_max(heatmap_batch, axis=(1, 2), keepdims=True)
        heatmap_batch = heatmap_batch / (max_vals + tf.keras.backend.epsilon())
        return heatmap_batch, preds

    def run_grad_cam_analysis_final(model, model_name, output_base_path, test_dataset):
        print(f"\\n--- Bắt đầu phân tích cho mô hình: {model_name} ---")
        results_by_class = { name: {'correct_heatmaps': [], 'correct_confidences': [], 'correct_images': [],
                                    'incorrect_heatmaps': [], 'incorrect_confidences': [], 'incorrect_images': []}
                            for name in target_names }

        print("  - Xử lý các batch trên TPU...")
        for images_batch, labels_batch in tqdm(test_dataset, desc=f"Analyzing {model_name}"):
            heatmap_batch, preds_batch = get_grad_cam_batched(model, images_batch)
            y_pred_probs_batch = tf.nn.softmax(preds_batch).numpy()
            y_pred_batch = np.argmax(y_pred_probs_batch, axis=1)
            y_true_batch = np.argmax(labels_batch.numpy(), axis=1)

            for i in range(images_batch.shape[0]):
                y_pred, y_true = y_pred_batch[i], y_true_batch[i]
                true_class_name = target_names[y_true]
                
                if y_pred == y_true:
                    results_by_class[true_class_name]['correct_heatmaps'].append(heatmap_batch[i].numpy())
                    results_by_class[true_class_name]['correct_confidences'].append(y_pred_probs_batch[i, y_pred])
                    results_by_class[true_class_name]['correct_images'].append(images_batch[i].numpy())
                else:
                    results_by_class[true_class_name]['incorrect_heatmaps'].append(heatmap_batch[i].numpy())
                    results_by_class[true_class_name]['incorrect_confidences'].append(y_pred_probs_batch[i, y_pred])
                    results_by_class[true_class_name]['incorrect_images'].append(images_batch[i].numpy())

        print("  - Đang lưu ảnh Grad-CAM và ảnh spectrogram cho các mẫu tiêu biểu...")
        for class_name in target_names:
            class_output_path = os.path.join(output_base_path, class_name)
            os.makedirs(class_output_path, exist_ok=True)
            class_results = results_by_class[class_name]

            if class_results['correct_confidences']:
                best_idx = np.argmax(class_results['correct_confidences'])
                image = class_results['correct_images'][best_idx]
                heatmap = class_results['correct_heatmaps'][best_idx]
                overlay = overlay_grad_cam(image[:, :, 0], heatmap)
                gradcam_filename = f"{model_name}_{class_name}_exemplar_correct_gradcam.png"
                plt.imsave(os.path.join(class_output_path, gradcam_filename), overlay)
                spectrogram_filename = f"{model_name}_{class_name}_exemplar_correct_spectrogram.png"
                plt.imsave(os.path.join(class_output_path, spectrogram_filename), image[:, :, 0], cmap='viridis')

            if class_results['incorrect_confidences']:
                worst_idx = np.argmax(class_results['incorrect_confidences'])
                image = class_results['incorrect_images'][worst_idx]
                heatmap = class_results['incorrect_heatmaps'][worst_idx]
                overlay = overlay_grad_cam(image[:, :, 0], heatmap)
                gradcam_filename = f"{model_name}_{class_name}_exemplar_incorrect_gradcam.png"
                plt.imsave(os.path.join(class_output_path, gradcam_filename), overlay)
                spectrogram_filename = f"{model_name}_{class_name}_exemplar_incorrect_spectrogram.png"
                plt.imsave(os.path.join(class_output_path, spectrogram_filename), image[:, :, 0], cmap='viridis')

    # --- VÒNG LẶP CHÍNH ĐỂ PHÂN TÍCH 5 FOLDS ---
    teacher_models_main_path = os.path.join(grad_cam_main_path, "teacher_models")
    os.makedirs(teacher_models_main_path, exist_ok=True)

    for fold_number in range(1, N_SPLITS + 1):
        print(f"\\n---> Bắt đầu phân tích Grad-CAM cho Fold {fold_number}/{N_SPLITS}...")
        model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        
        if not os.path.exists(model_path):
            print(f"Bỏ qua Fold {fold_number}, không tìm thấy file: {model_path}")
            continue
        
        try:
            with strategy.scope():
                teacher_model = tf.keras.models.load_model(
                    model_path,
                    custom_objects={
                        'focal_loss_fixed': focal_loss_from_logits_optimized(alpha=alpha_weights_list),
                        'MacroF1Score': MacroF1Score 
                    }
                )
            
            model_name = f"fold_{fold_number}"
            fold_output_path = os.path.join(teacher_models_main_path, model_name)
            os.makedirs(fold_output_path, exist_ok=True)
            
            # Gọi hàm phân tích với dataset đã được chuẩn bị
            run_grad_cam_analysis_final(teacher_model, model_name, fold_output_path, test_ds_batched)
        
        except Exception as e:
            print(f"!!! Lỗi khi phân tích Grad-CAM cho Fold {fold_number}: {e}")

    print("\\n--- Toàn bộ quá trình phân tích Grad-CAM đã hoàn tất ---")

In [ ]:
# --- BƯỚC TỐI ƯU & CHUYỂN ĐỔI CẢ 5 MÔ HÌNH SANG TFLITE (POST-TRAINING) ---

print("--- Bắt đầu quy trình chuyển đổi 5-Fold sang TFLite ---")

# Kiểm tra xem quá trình huấn luyện đã hoàn tất và có đủ kết quả chưa
if 'fold_f1s' in locals() and len(fold_f1s) == N_SPLITS:
    
    # Lấy thông tin chia fold một lần để tái sử dụng
    skf_split = list(skf.split(train_val_df, y_labels_for_split, groups_for_split))

    # === BẮT ĐẦU VÒNG LẶP QUA 5 FOLDS ===
    for fold_index in range(N_SPLITS):
        fold_number = fold_index + 1
        print("=" * 60)
        print(f"--- Bắt đầu chuyển đổi cho Fold {fold_number} ---")
        
        # 1. Xác định đường dẫn cho fold hiện tại
        SAVED_MODEL_PATH = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}')
        TFLITE_MODEL_PATH = os.path.join(KAGGLE_OUTPUT_PATH, f'model_fold_{fold_number}_quantized.tflite')

        if not os.path.exists(SAVED_MODEL_PATH):
            print(f"Lỗi: Không tìm thấy thư mục model tại '{SAVED_MODEL_PATH}'. Bỏ qua fold này.")
            continue

        # 2. Tạo Representative Dataset từ tập validation của CHÍNH FOLD NÀY
        print(f"Đang tạo representative dataset cho Fold {fold_number}...")
        _, val_indices = skf_split[fold_index]
        val_df_for_calib = train_val_df.iloc[val_indices]

        def representative_data_gen():
            for _, row in val_df_for_calib.sample(n=min(150, len(val_df_for_calib)), random_state=SEED).iterrows():
                spectrogram = np.load(row['filepath']).astype(np.float32)
                
                image_tensor = tf.convert_to_tensor(spectrogram)
                image_3d = tf.stack([image_tensor]*3, axis=-1)
                image_resized = tf.image.resize(image_3d, [INPUT_SHAPE[0], INPUT_SHAPE[1]])
                min_val = tf.reduce_min(image_resized)
                max_val = tf.reduce_max(image_resized)
                image_scaled_01 = (image_resized - min_val) / (max_val - min_val + 1e-7)
                image_scaled_255 = image_scaled_01 * 255.0
                image_preprocessed = preprocess_input(image_scaled_255)
                
                yield [tf.expand_dims(image_preprocessed, axis=0)]

        # 3. Chuyển đổi và Lượng tử hóa
        print(f"Đang chuyển đổi mô hình của Fold {fold_number}...")
        converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_data_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        
        tflite_quant_model = converter.convert()

        # 4. Lưu file TFLite
        with open(TFLITE_MODEL_PATH, 'wb') as f:
            f.write(tflite_quant_model)
        
        print(f"Đã lưu thành công model TFLite cho Fold {fold_number} tại: {TFLITE_MODEL_PATH}")
        print(f"Kích thước file: {len(tflite_quant_model) / (1024 * 1024):.2f} MB")

    print("=" * 60)
    print("\\n Hoàn tất chuyển đổi cho cả 5 mô hình!")

else:
    print("Lỗi: Không tìm thấy kết quả của 5 fold ('fold_f1s'). Vui lòng chạy ô huấn luyện trước khi chạy ô này.")